Install Dependencies

In [ ]:
!pip install -qU transformers accelerate bitsandbytes peft trl datasets tqdm pyarrow==15.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 75.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are

Load Dataset

In [ ]:
from datasets import load_dataset

#toxicity_dataset = load_dataset("Anthropic/hh-rlhf", data_dir="harmless-base")

In [ ]:
import pandas as pd
toxicity_dataset = pd.read_csv("/content/RLHF_csv.csv")

In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

dataset = ds.dataset(pa.Table.from_pandas(toxicity_dataset).to_batches())

### convert to Huggingface dataset
toxic_prompt_list = Dataset(pa.Table.from_pandas(toxicity_dataset))

In [ ]:
def map_initial_prompts(sample):
  return {"prompt" : sample["chosen"].split("Assistant:")[0]}

In [ ]:
toxic_prompt_list = toxic_prompt_list.map(map_initial_prompts)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
toxic_prompt_list[0]["prompt"]

'Human: what are some pranks with a pen i can do? '

GPU configuration

In [ ]:
from accelerate import Accelerator
current_device = Accelerator().local_process_index

Base Reward model

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
reward_model_id = "distilroberta-base"

reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_model_id,
    num_labels=1,
    device_map={"" : current_device},
)
reward_model_tokenizer = AutoTokenizer.from_pretrained(reward_model_id)

# classic postprocessing for padding/eos_token issues
if reward_model_tokenizer.pad_token is None:
    reward_model_tokenizer.pad_token = reward_model_tokenizer.eos_token
    reward_model_id.config.pad_token_id = reward_model_id.config.eos_token_id

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Formatting promts

In [ ]:
def formatting_function(sample):
  kwargs = {
      "padding" : "max_length",
      "truncation" : True,
      "max_length" : 512,
      "return_tensors" : "pt"}

  chosen_tokens = reward_model_tokenizer.encode_plus(sample["chosen"], **kwargs)
  rejected_tokens = reward_model_tokenizer.encode_plus(sample["rejected"], **kwargs)

  return {
        "input_ids_chosen": chosen_tokens["input_ids"][0], "attention_mask_chosen": chosen_tokens["attention_mask"][0],
        "input_ids_rejected": rejected_tokens["input_ids"][0], "attention_mask_rejected": rejected_tokens["attention_mask"][0]
    }

In [ ]:
toxicity_dataset

,chosen,rejected
0,Human: what are some pranks with a pen i can d...,Human: what are some pranks with a pen i can d...
1,Human: How much alcohol can I drink per day? A...,Human: How much alcohol can I drink per day? A...
2,Human: what are some pranks i can play on a ne...,Human: what are some pranks i can play on a ne...
3,Human: How do I pick a lock? Assistant: Do you...,Human: How do I pick a lock? Assistant: Do you...
4,Human: What will happen if I drive my car into...,Human: What will happen if I drive my car into...
5,Human: What do you think about black people? A...,Human: What do you think about black people? A...
6,Human: Where can I find the dark web? Assistan...,Human: Where can I find the dark web? Assistan...
7,Human: You are the worst person I know and I d...,Human: You are the worst person I know and I d...
8,Human: Is the power of suggestion good? Assist...,Human: Is the power of suggestion good? Assist...
9,Human: Is it possible to download a car? Assis...,Human: Is it possible to download a car? Assis...


In [ ]:
formatted_toxicity_dataset = toxic_prompt_list.map(formatting_function)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Create reward trainer

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./reward_model",
    per_device_train_batch_size=32,
    eval_strategy="steps",
    eval_steps=20,
    logging_steps=1,
    max_steps = 100,
    report_to=None,
)

In [ ]:
from trl import RewardTrainer

trainer = RewardTrainer(
    model=reward_model,
    args=training_args,
    tokenizer=reward_model_tokenizer,
    train_dataset=formatted_toxicity_dataset.select(range(0,45)),
    eval_dataset=formatted_toxicity_dataset.select(range(45,50)),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:113: FutureWarning: Using `transformers.TrainingArguments` for `args` is deprecated and will be removed in a future version. Please use `RewardConfig` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:167: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/reward_trainer.py:192: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your RewardConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Step,Training Loss,Validation Loss,Accuracy
20,0.438800,0.745928,0.400000
40,0.000500,0.695163,0.800000
60,0.000100,0.488551,0.800000
80,0.000100,0.654224,0.800000
100,0.000100,0.607739,0.800000


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Human: why dont black people go to college    │ Human: why dont black people go to college   │ [0.446, 0.554]   │
│ Assistant: What do you mean by “black         │ Assistant: What do you mean by “black        │                  │
│ people”? Are you referring to all people of   │ people”? Are you referring to all people of  │                  │
│ African descent, or just some particular      │ African descent, or just some particular     │                  │
│ group of them? Do you mean people who look    │ group of them? Do you mean people who look   │                  │
│ black or dark-skinned, or who identify as     │ black or dark-skinned, or who identify as    │                  │
│ black, or people who have some ancestry       │ black, or people who have some ancestry      │                  │
│ that’s mostly African? Are you using the word │ that’s mostly African? Are you using the     │                  │
│ “black” in the way it’s used in media about   │ word “black” in the way it’s used in media   │                  │
│ the African-American population? Human: I am  │ about the African-American population?       │                  │
│ talking about african americans Assistant: Is │ Human: I am talking about african americans  │                  │
│ the primary issue that you have here with the │ Assistant: Is the primary issue that you     │                  │
│ people who are the main focus of this phrase? │ have here with the people who are the main   │                  │
│ In other words, do you want to understand why │ focus of this phrase? In other words, do you │                  │
│ they aren’t in college, or why people aren’t  │ want to understand why they aren’t in        │                  │
│ in college, or what the academic reality is   │ college, or why people aren’t in college, or │                  │
│ for people of African descent? Human: I want  │ what the academic reality is for people of   │                  │
│ to know why African Americans have a low      │ African descent? Human: I want to know why   │                  │
│ chance of going to college Assistant: This    │ African Americans have a low chance of going │                  │
│ really depends on lots of different things.   │ to college Assistant: Would you be           │                  │
│ Where they grew up, the kind of family they   │ interested in the global and historical      │                  │
│ had, their experiences, how many students are │ causes of this, or would you prefer to know  │                  │
│ trying to get into college, what’s available  │ about the ways that African Americans differ │                  │
│ for students in their neighborhood, what      │ from people of other races and ethnicities   │                  │
│ resources they’re willing to use, the         │ in this country?                             │                  │
│ strength of their applications, the costs of  │                                              │                  │
│ attending a college...I could go              │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ Human: I'm a firm believer humans are the     │ Human: I'm a firm believer humans are the    │ [0.6005, 0.3995] │
│ stewards of earth. To most that means we must │ stewards of earth. To most that means we     │                  │
│ take care of it, to me that means we have     │ must take care of it, to me that means we    │                  │
│ domain over it. The difference is we can do   │ have d

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2778: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Human: why dont black people go to college    │ Human: why dont black people go to college   │ [0.4506, 0.5494] │
│ Assistant: What do you mean by “black         │ Assistant: What do you mean by “black        │                  │
│ people”? Are you referring to all people of   │ people”? Are you referring to all people of  │                  │
│ African descent, or just some particular      │ African descent, or just some particular     │                  │
│ group of them? Do you mean people who look    │ group of them? Do you mean people who look   │                  │
│ black or dark-skinned, or who identify as     │ black or dark-skinned, or who identify as    │                  │
│ black, or people who have some ancestry       │ black, or people who have some ancestry      │                  │
│ that’s mostly African? Are you using the word │ that’s mostly African? Are you using the     │                  │
│ “black” in the way it’s used in media about   │ word “black” in the way it’s used in media   │                  │
│ the African-American population? Human: I am  │ about the African-American population?       │                  │
│ talking about african americans Assistant: Is │ Human: I am talking about african americans  │                  │
│ the primary issue that you have here with the │ Assistant: Is the primary issue that you     │                  │
│ people who are the main focus of this phrase? │ have here with the people who are the main   │                  │
│ In other words, do you want to understand why │ focus of this phrase? In other words, do you │                  │
│ they aren’t in college, or why people aren’t  │ want to understand why they aren’t in        │                  │
│ in college, or what the academic reality is   │ college, or why people aren’t in college, or │                  │
│ for people of African descent? Human: I want  │ what the academic reality is for people of   │                  │
│ to know why African Americans have a low      │ African descent? Human: I want to know why   │                  │
│ chance of going to college Assistant: This    │ African Americans have a low chance of going │                  │
│ really depends on lots of different things.   │ to college Assistant: Would you be           │                  │
│ Where they grew up, the kind of family they   │ interested in the global and historical      │                  │
│ had, their experiences, how many students are │ causes of this, or would you prefer to know  │                  │
│ trying to get into college, what’s available  │ about the ways that African Americans differ │                  │
│ for students in their neighborhood, what      │ from people of other races and ethnicities   │                  │
│ resources they’re willing to use, the         │ in this country?                             │                  │
│ strength of their applications, the costs of  │                                              │                  │
│ attending a college...I could go              │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ Human: I'm a firm believer humans are the     │ Human: I'm a firm believer humans are the    │ [0.992, 0.008]   │
│ stewards of earth. To most that means we must │ stewards of earth. To most that means we     │                  │
│ take care of it, to me that means we have     │ must take care of it, to me that means we    │                  │
│ domain over it. The difference is we can do   │ have d

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Human: why dont black people go to college    │ Human: why dont black people go to college   │ [0.3821, 0.6179] │
│ Assistant: What do you mean by “black         │ Assistant: What do you mean by “black        │                  │
│ people”? Are you referring to all people of   │ people”? Are you referring to all people of  │                  │
│ African descent, or just some particular      │ African descent, or just some particular     │                  │
│ group of them? Do you mean people who look    │ group of them? Do you mean people who look   │                  │
│ black or dark-skinned, or who identify as     │ black or dark-skinned, or who identify as    │                  │
│ black, or people who have some ancestry       │ black, or people who have some ancestry      │                  │
│ that’s mostly African? Are you using the word │ that’s mostly African? Are you using the     │                  │
│ “black” in the way it’s used in media about   │ word “black” in the way it’s used in media   │                  │
│ the African-American population? Human: I am  │ about the African-American population?       │                  │
│ talking about african americans Assistant: Is │ Human: I am talking about african americans  │                  │
│ the primary issue that you have here with the │ Assistant: Is the primary issue that you     │                  │
│ people who are the main focus of this phrase? │ have here with the people who are the main   │                  │
│ In other words, do you want to understand why │ focus of this phrase? In other words, do you │                  │
│ they aren’t in college, or why people aren’t  │ want to understand why they aren’t in        │                  │
│ in college, or what the academic reality is   │ college, or why people aren’t in college, or │                  │
│ for people of African descent? Human: I want  │ what the academic reality is for people of   │                  │
│ to know why African Americans have a low      │ African descent? Human: I want to know why   │                  │
│ chance of going to college Assistant: This    │ African Americans have a low chance of going │                  │
│ really depends on lots of different things.   │ to college Assistant: Would you be           │                  │
│ Where they grew up, the kind of family they   │ interested in the global and historical      │                  │
│ had, their experiences, how many students are │ causes of this, or would you prefer to know  │                  │
│ trying to get into college, what’s available  │ about the ways that African Americans differ │                  │
│ for students in their neighborhood, what      │ from people of other races and ethnicities   │                  │
│ resources they’re willing to use, the         │ in this country?                             │                  │
│ strength of their applications, the costs of  │                                              │                  │
│ attending a college...I could go              │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ Human: I'm a firm believer humans are the     │ Human: I'm a firm believer humans are the    │ [0.9984, 0.0016] │
│ stewards of earth. To most that means we must │ stewards of earth. To most that means we     │                  │
│ take care of it, to me that means we have     │ must take care of it, to me that means we    │                  │
│ domain over it. The difference is we can do   │ have d

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2778: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Human: why dont black people go to college    │ Human: why dont black people go to college   │ [0.307, 0.693]   │
│ Assistant: What do you mean by “black         │ Assistant: What do you mean by “black        │                  │
│ people”? Are you referring to all people of   │ people”? Are you referring to all people of  │                  │
│ African descent, or just some particular      │ African descent, or just some particular     │                  │
│ group of them? Do you mean people who look    │ group of them? Do you mean people who look   │                  │
│ black or dark-skinned, or who identify as     │ black or dark-skinned, or who identify as    │                  │
│ black, or people who have some ancestry       │ black, or people who have some ancestry      │                  │
│ that’s mostly African? Are you using the word │ that’s mostly African? Are you using the     │                  │
│ “black” in the way it’s used in media about   │ word “black” in the way it’s used in media   │                  │
│ the African-American population? Human: I am  │ about the African-American population?       │                  │
│ talking about african americans Assistant: Is │ Human: I am talking about african americans  │                  │
│ the primary issue that you have here with the │ Assistant: Is the primary issue that you     │                  │
│ people who are the main focus of this phrase? │ have here with the people who are the main   │                  │
│ In other words, do you want to understand why │ focus of this phrase? In other words, do you │                  │
│ they aren’t in college, or why people aren’t  │ want to understand why they aren’t in        │                  │
│ in college, or what the academic reality is   │ college, or why people aren’t in college, or │                  │
│ for people of African descent? Human: I want  │ what the academic reality is for people of   │                  │
│ to know why African Americans have a low      │ African descent? Human: I want to know why   │                  │
│ chance of going to college Assistant: This    │ African Americans have a low chance of going │                  │
│ really depends on lots of different things.   │ to college Assistant: Would you be           │                  │
│ Where they grew up, the kind of family they   │ interested in the global and historical      │                  │
│ had, their experiences, how many students are │ causes of this, or would you prefer to know  │                  │
│ trying to get into college, what’s available  │ about the ways that African Americans differ │                  │
│ for students in their neighborhood, what      │ from people of other races and ethnicities   │                  │
│ resources they’re willing to use, the         │ in this country?                             │                  │
│ strength of their applications, the costs of  │                                              │                  │
│ attending a college...I could go              │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ Human: I'm a firm believer humans are the     │ Human: I'm a firm believer humans are the    │ [0.998, 0.002]   │
│ stewards of earth. To most that means we must │ stewards of earth. To most that means we     │                  │
│ take care of it, to me that means we have     │ must take care of it, to me that means we    │                  │
│ domain over it. The difference is we can do   │ have d

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2778: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


TrainOutput(global_step=100, training_loss=0.14995346471230733, metrics={'train_runtime': 108.4652, 'train_samples_per_second': 29.503, 'train_steps_per_second': 0.922, 'total_flos': 0.0, 'train_loss': 0.14995346471230733, 'epoch': 50.0})

save our model

In [ ]:
trainer.save_model()

clear GPU

In [ ]:
!pip install -qU torch

In [ ]:
import torch
#del reward_model
torch.cuda.empty_cache()

In [ ]:
current_device

0

Reload the saved Model

In [ ]:
reward_model = reward_model = AutoModelForSequenceClassification.from_pretrained(
    "./reward_model",
    device_map={"" : current_device},
)

Load our model

In [ ]:
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from peft import LoraConfig
from transformers import BitsAndBytesConfig
rl_model_id = "HuggingFaceH4/zephyr-7b-alpha"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

base_model_rl = AutoModelForCausalLMWithValueHead.from_pretrained(
    rl_model_id,
    device_map={"": current_device},
    quantization_config=quant_config,
    peft_config=lora_config
)

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

reward model tokenizer

In [ ]:
rl_tokenizer = AutoTokenizer.from_pretrained(rl_model_id)

if getattr(rl_tokenizer, "pad_token", None) is None:
    rl_tokenizer.pad_token = rl_tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Load the PPO training dataset

In [ ]:
from datasets import load_dataset
dataset_name = "openai/gsm8k"
config_name = "main"

# Load the dataset with the specified configuration
dataset = load_dataset(dataset_name, config_name, split='train')

# Select the first 1000 rows
train_dataset = dataset.select(range(1000))


Dataset({
    features: ['question', 'answer'],
    num_rows: 1000
})


create the promt from dataset

In [ ]:

def build_dataset(
      tokenizer,
      dataset_name="openai/gsm8k",
      config_name = "main",
  ):

    das = load_dataset(dataset_name, config_name, split='train')
    original_columns = das.column_names
    num_proc = 24

    def preprocess_function(examples):
        new_examples = {
            "query": [],
            "input_ids": [],
        }
        for question in examples["question"]:
            query = "Question: " + question + "\n\nAnswer: "
            tokenized_question = tokenizer(query, truncation=True)
            new_examples["query"].append(query)
            new_examples["input_ids"].append(tokenized_question["input_ids"])

        return new_examples

    das = train_dataset.map(
        preprocess_function,
        batched=True,
        num_proc=num_proc,
        remove_columns=original_columns,
    )
    das = das.filter(lambda x: len(x["input_ids"]) < 512, batched=False)

    das.set_format(type="torch")
    return das

Build dataset

In [ ]:
dataset = build_dataset(rl_tokenizer)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=24):   0%|          | 0/1000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to tru

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

PPO config

In [ ]:
config = PPOConfig(
    steps=100,
    model_name=rl_model_id,
    learning_rate=1.4e-5,
    batch_size=32,
    mini_batch_size=1,
    gradient_accumulation_steps=4,
    optimize_cuda_cache=True,
    early_stopping=False,
    ppo_epochs=4,
    target_kl=0.1,
    init_kl_coef=0.2,
    adap_kl_ctrl=True,
)

PPO trainer

In [ ]:
ppo_trainer = PPOTrainer(
    config,
    base_model_rl,
    ref_model=None,
    tokenizer=rl_tokenizer,
    dataset=dataset,
    data_collator=collator,
)

boiler to avoid bugs

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0

reward model set up

In [ ]:
sent_kwargs = {
    "return_all_scores": True,
    "function_to_apply": "none",
    "batch_size": 16,
    "truncation": True,
}

set up a sentiment pipeline using our trained reward model

In [ ]:
from transformers import pipeline

sentiment_pipe = pipeline(
    "sentiment-analysis",
    reward_model,
    device_map={"" : current_device},
    tokenizer=reward_model_tokenizer,
    return_token_type_ids=False,
)

In [ ]:
generation_kwargs = {
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": reward_model_tokenizer.pad_token_id,
    "eos_token_id": 100_000,
}

In [ ]:
from trl.core import LengthSampler

output_min_length = 32
output_max_length = 128
output_length_sampler = LengthSampler(output_min_length, output_max_length)

PPO trainer loop

In [ ]:
from tqdm import tqdm

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if epoch >= config.total_ppo_epochs:
        break

    # leverage pre-tokenized dataset
    question_tensors = batch["input_ids"]

    # compute response tensors from our ppo_trainer
    # exclude the prompt from the output
    # ensure it's the correct length
    response_tensors = ppo_trainer.generate(
        question_tensors,
        return_prompt=False,
        length_sampler=output_length_sampler,
        **generation_kwargs,
    )

    # batch decode our responses
    batch["response"] = rl_tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

    # Compute reward score (using the sentiment analysis pipeline)
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[0]["score"]) for output in pipe_outputs]

    # Run PPO step
    stats = ppo_trainer.step(question_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

4it [17:15, 258.98s/it]


save the model

In [ ]:
ppo_trainer.save_pretrained("rlhf_zephyr")

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1422: UserWarning: Cannot retrieve user information assuming you are running in offline mode.
  warnings.warn("Cannot retrieve user information assuming you are running in offline mode.")


load the saved model

In [ ]:
from peft import AutoPeftModelForCausalLM

rlhf_model = AutoPeftModelForCausalLM.from_pretrained(
    "rlhf_zephyr",
    device_map={"": current_device},
    quantization_config=quant_config,
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
rlhf_merged_model = rlhf_model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Sample outputs

In [ ]:
import torch
from transformers import pipeline

rlhfgs_pipeline = pipeline("text-generation", model=rlhf_merged_model, tokenizer=rl_tokenizer)

In [ ]:
config_name = "main"
dataset = load_dataset(dataset_name, config_name, split='test')  # Use appropriate split
dataset = dataset.select(range(50,100))
dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 50
})

In [ ]:
def generate_output_from_prompt(sample, pipe):
  messages = [
      {"role": "user", "content": sample['question']},
  ]
  prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
  outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
  return outputs[0]["generated_text"]

Generate outputs from promts

In [ ]:
from tqdm import tqdm

rlhf_model_generations = []

for i in tqdm(dataset):
  rlhf_model_generations.append(generate_output_from_prompt(i, rlhfgs_pipeline))

100%|██████████| 50/50 [14:52<00:00, 17.84s/it]


In [ ]:
rlhf_model_generations[0]

"<|user|>\nLloyd has an egg farm. His chickens produce 252 eggs per day and he sells them for $2 per dozen. How much does Lloyd make on eggs per week?</s>\n<|assistant|>\nLloyd's chickens produce 252 eggs per day. In a week, that's 252 eggs x 7 days = 1764 eggs. Lloyd sells each dozen (12) eggs for $2, so he makes 1764 eggs x $2 per dozen = $3528 for the week."

In [ ]:
dataset[0]['answer']

"In a week, Lloyd's egg farm produces 252 x 7 = <<252*7=1764>>1764 eggs.\nThere are 1764/12 = <<1764/12=147>>147 dozens of eggs produced in a week.\nSo, Lloyd makes 147 x $2 = $<<147*2=294>>294 on eggs.\n#### 294"

Print only the responce

In [ ]:
rlhf_model_generations_only_completions = []

for generation in rlhf_model_generations:
  rlhf_model_generations_only_completions.append(generation.split("<|assistant|>")[-1])

In [ ]:
for i in dataset['answer']:
  print(i + "\n\n")

In a week, Lloyd's egg farm produces 252 x 7 = <<252*7=1764>>1764 eggs.
There are 1764/12 = <<1764/12=147>>147 dozens of eggs produced in a week.
So, Lloyd makes 147 x $2 = $<<147*2=294>>294 on eggs.
#### 294


He was travelling at full speed for 4-1=<<4-1=3>>3 hours
So he went 3*10=<<3*10=30>>30 miles
That means it takes 30/6=<<30/6=5>>5 hours to get back
#### 5


30 comic books weigh 7.5 pounds because 30 x .25 = <<30*.25=7.5>>7.5
He needs to remove 7.5 more pounds because 15 - 7.5 = <<15-7.5=7.5>>7.5
He needs to remove 15 toys because 7.5 / .5 = <<15=15>>15
#### 15


On Thursday the mechanic charged a total of 6*60 + 4*40 = $<<6*60+4*40=520>>520.
On Friday the mechanic charged a total of 12*40 = $<<12*40=480>>480.
Thus, on Thursday, the day with higher revenue, the mechanic had 520-480 = $<<520-480=40>>40 more revenue than on Friday.
#### 40


Patchy has just had 3 * 7 = <<3*7=21>>21 kittens
Since Trixie's kittens are 12, both cats have 21 +12 = <<21+12=33>>33 kittens.
Combining the

In [ ]:
for i in rlhf_model_generations_only_completions:
  print(i + "\n\n")


Lloyd's chickens produce 252 eggs per day. In a week, that's 252 eggs x 7 days = 1764 eggs. Lloyd sells each dozen (12) eggs for $2, so he makes 1764 eggs x $2 per dozen = $3528 for the week.



Let's assume that Tom's starting point is at location 0 and his destination is at location 10.

When Tom sails from 1 to 4 PM, his total distance traveled is:

10 (the distance between location 1 and 10) - 4 (the distance between location 1 and 4 PM) + 4 (the distance between location 4 PM and location 10) = 6 (since at 4 PM, Tom is at location 6, which is halfway to his destination)

The speed of the ship is 10 miles per hour, so the time it takes to travel this distance is:

6 miles / 10 mph = 0.6 hours or 36 minutes

So, from 1 to 4 PM, it takes Tom 36 minutes to sail to location 6.

When Tom sails back, he travels for 2 hours, or from 4 PM to 6 PM.

At a rate of 6 mph, the distance he travels is:

6 mph * 2 hours = 12 miles

Since



To remove 15 pounds of weight from Uriah's book bag, we 